# Reinforcement Learning: Monte Carlo Prediction 

very kindly version

# Google Drive와 연동하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 본인의 파일 저장 경로로 변경해주세요
%cd /content/drive/MyDrive/'Colab Notebooks'/RL2023/MC_Prediction
!ls
import sys; sys.path.append('..') # add project root to the python path
import numpy as np
import matplotlib.pyplot as plt
from gridworld import *

np.random.seed(0)

## `GridWorld` 초기화하기

가로로 `nx` 개, 세로로 `ny` 개의 칸을 가진 `GridworldEnv`를 만듭니다!

In [ ]:
nx, ny = 5, 5
env = GridworldEnv([ny, nx])

def get_action(state): 현재 state에서 action을 정의하는 함수. 본 예제에서는 uniform random 으로 정의합니다. (즉, 모든 방향의 확률이 동일함)

In [ ]:
def get_action(state):

        action = np.random.choice(range(4))

        return action

###  `run_episode()`

MC 기법은 하나의 에피소드가 진행되고 state,action,reward 정보를 모두 모아 반환합니다.


In [ ]:
def run_episode(env, timeout=1000):
    env.reset()
    states = []
    actions = []
    rewards = []

    i = 0
    timeouted = False
    while True:
        state = env.s
        action = get_action(state)
        next_state, reward, done, info = env.step(action)
        states.append(state)
        actions.append(action)
        rewards.append(reward)

        if done:
            break
        else:
            i += 1
            if i >= timeout:
                timeouted = True
                break

    if not timeouted:
        episode = (states, actions, rewards)
    return episode

no_episode 개를 생성하여 episodes 에 저장합니다.

기본 1000개이지만 샘플 숫자를 바꿔가면서 해보세요.

In [ ]:
episodes=[]
no_episode = 10000
for _ in range(no_episode):
    episodes.append(run_episode(env))

에피소드의 형식 및 transit이 궁금하면 아래의 코드를 실행해보세요

episode는 s1, a1, r1, s2, a2, r2... 순으로 저장되어 있습니다.

In [ ]:
#  episodes 는 list 이므로, len으로 크기 확인
print(len(episodes))
# 10개만 출력해봅시다
print(episodes[1:10])


# MC
$$V(s) \leftarrow \frac{S(s)}{N(s)}$$
$$Q(s,a) \leftarrow \frac{S(s,a)}{N(s,a)}$$

$S(s)$ : 상태 s 에 대한 return들의 합 \\
$N(s)$ : 상태 s 를 방문한 횟수 \\
$S(s,a)$ : 상태 s에서 a 행동을 했을 때 return들의 값 \\
$N(s,a)$ : 상태 s에서 a 행동을 한 횟수

Every Visit 방식입니다.

In [ ]:
s_v=np.zeros(shape=nx*ny)
s_q = np.zeros(shape=(nx*ny,4))
n_v=np.zeros(shape=nx*ny)
n_q = np.zeros(shape=(nx*ny,4))
gamma=1.0
lr=1e-3

for episode in episodes:
    states, actions, rewards = episode

    # reversing the inputs!
    # for efficient computation of returns
    states = reversed(states)
    actions = reversed(actions)
    rewards = reversed(rewards)

    iter = zip(states, actions, rewards)
    cum_r = 0
    for s, a, r in iter:
        cum_r *= gamma
        cum_r += r

        n_v[s] += 1
        n_q[s, a] += 1

        s_v[s] += cum_r
        s_q[s, a] += cum_r
v = s_v / (n_v)
q = s_q / (n_q)

## 추산된 상태가치함수 $V(s)$ 및 $Q(s,a)$ 확인하기

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,6))
visualize_value_function(ax[0], v, nx, ny)
_ = ax[0].set_title("Value pi")
visualize_policy(ax[1], q, nx, ny)
_ = ax[1].set_title("Greedy policy")

In [ ]:
import pandas as pd

def highlight_max(s):
    '''
    highlight the maximum in a Series green.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

def visualize_q(q):
    df = pd.DataFrame(q, columns=['up', 'right', 'down', 'left']).T
    df = df.style.apply(highlight_max)
    return df
df = visualize_q(q)
df

## Incremental MC (Mission)


$$V(s) \leftarrow V(s) + \alpha (G(s) - V(s))$$
$$Q(s,a) \leftarrow Q(s,a) + \alpha (G(s,a) - Q(s,a))$$

$\alpha$: learning rate \\
$G(s)$:상태 s에서의 return \\
$G(s,a)$: 상태 s, 행동 a에서의 return

alpha = 값을 정의한 후 incremental MC 방식으로 v(s)와 q(s,a)의 값을 계산해보세요

In [ ]:
v=np.zeros(shape=nx*ny)
q = np.zeros(shape=(nx*ny,4))
gamma=1.0
alpha=1e-3

# ------------------- MISSION ---------------------
# 여기에 코드를 작성하세요. 위의 코드를 이용하세요.

## 추산된 상태가치함수 $V(s)$ 및 $Q(s,a)$ 확인하기

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,6))
visualize_value_function(ax[0], v, nx, ny)
_ = ax[0].set_title("Value pi")
visualize_policy(ax[1], q, nx, ny)
_ = ax[1].set_title("Greedy policy")

In [ ]:
import pandas as pd

def highlight_max(s):
    '''
    highlight the maximum in a Series green.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]

def visualize_q(q):
    df = pd.DataFrame(q, columns=['up', 'right', 'down', 'left']).T
    df = df.style.apply(highlight_max)
    return df
df = visualize_q(q)
df